In [24]:
from nltk.corpus import wordnet as wn

In [63]:
wn.lemmas('break',pos='n')

[Lemma('interruption.n.02.break'),
 Lemma('break.n.02.break'),
 Lemma('fault.n.04.break'),
 Lemma('rupture.n.02.break'),
 Lemma('respite.n.02.break'),
 Lemma('breakage.n.03.break'),
 Lemma('pause.n.01.break'),
 Lemma('fracture.n.01.break'),
 Lemma('break.n.09.break'),
 Lemma('break.n.10.break'),
 Lemma('break.n.11.break'),
 Lemma('break.n.12.break'),
 Lemma('break.n.13.break'),
 Lemma('break.n.14.break'),
 Lemma('open_frame.n.01.break'),
 Lemma('break.n.16.break')]

In [110]:
l1=wn.lemmas('break',pos='n')[1]

In [111]:
s1=l1.synset()
s1

Synset('break.n.02')

In [114]:
s1.lemmas()[0].count()

2

In [74]:
a=s1.definition()
b=s1.examples()
a+" "+b[0]

'an unexpected piece of good luck he finally got his big break'

In [25]:
from nltk.corpus import stopwords

In [69]:
def tokenize(s): 
    """
    a naive tokenizer that splits on punctuation and whitespaces.  
    """
    s = "".join(" " if x in string.punctuation else x for x in s.lower())    
    return s.split() 

In [26]:
def get_candidates(lemma, pos) :#-> List[str]:
    ls=wn.lemmas(lemma,pos)
    result=[]
    for l in ls:
        for m in l.synset().lemmas():
            if m.name()!=lemma:
                result.append(str(m.name()))
    return list(set(result))

In [18]:
get_candidates('slow','a')

['dull',
 'dumb',
 'wearisome',
 'dim',
 'obtuse',
 'deadening',
 'ho-hum',
 'sluggish',
 'boring',
 'dense',
 'tiresome',
 'tedious',
 'irksome']

In [61]:
def wn_frequency_predictor(lem,pos):# : Context)# -> str:
    lem=context.lemma
    pos=context.pos
    
    ls=wn.lemmas(lem,pos)
    result=dict()
    for l in ls:
        for m in l.synset().lemmas():
            if m.name()!=lem:
                if m.name() not in result.keys():
                    result[m.name()]=l.count()
                else:
                    result[m.name()]+=l.count()
                
    
    return [k for k,v in result.items() if v == max(result.values())][0]

In [62]:
lswn_frequency_predictor('slow','a')

'dense'

['dense', 'dim', 'dull', 'dumb', 'obtuse']

In [48]:
ls[4].synset().lemmas()

[Lemma('boring.s.01.boring'),
 Lemma('boring.s.01.deadening'),
 Lemma('boring.s.01.dull'),
 Lemma('boring.s.01.ho-hum'),
 Lemma('boring.s.01.irksome'),
 Lemma('boring.s.01.slow'),
 Lemma('boring.s.01.tedious'),
 Lemma('boring.s.01.tiresome'),
 Lemma('boring.s.01.wearisome')]

In [42]:
ls=wn.lemmas('slow',pos='a')
result=dict()
for l in ls:
    print(l.count())
    for m in l.synset().lemmas():
        if m.name()!='slow':
            if m.name() not in result.keys():
                result[m.name()]=l.count()
            else:
                result[m.name()]+=l.count()

23
3
1
0
0
0


In [109]:
a=[1,2,3]
b=[3,4,5]
# c=a.extend(b)
# a.remove(1)
#b
[i for i in b if not i in a and i !=4]

[5]

In [162]:
def wn_simple_lesk_predictor(lemma,pos,left_context,right_context):# : Context) -> str:
    lemma=context.lemma
    pos=context.pos
    ls=wn.lemmas(lemma,pos)
    left_context = context.left_context
    right_context =context.right_context
    left_context.extend(right_context)
    stop_words = stopwords.words('english')
    left_context=[i for i in left_context if not i in stop_words]
    overlap=dict()
    syn_freq=dict()
    for l in ls:
        syn_freq[l.synset()]=l.count()
        # construct definition
        s=l.synset()
        defi=s.definition()
        ex=s.examples()
        for e in ex:
            defi=defi+" "+e
        hyper=s.hypernyms()
        for h in hyper:
            defi=defi+" "+h.definition()
            for e in h.examples:
                defi=defi+" "+e
        # compute overlap
        defi=tokenize(defi)
        defi=[i for i in defi if not i in stop_words]
        count=0
        for d in defi:
            if d in left_context:
                count+=1
        overlap[s]=count
        
    #choose synset
    re=[k for k,v in overlap.items() if v == max(overlap.values()) and v!=0]
    if len(re)==0 or len(re)!=1:
        syn=[k for k,v in syn_freq.items()]
    else:
        syn=re
    
    #choose lexeme
    lexme=dict()
    for s in syn:
        for l in s.lemmas():
            lexme[l]=l.count()
            
    result=[k for k,v in lexme.items() if k.name()!=lemma]
    result=[k for k,v in lexme.items() if v == max(lexme.values())]
    result=result[0].name()
    
    return result
        




In [163]:
import string

In [164]:
syn_freq=wn_simple_lesk_predictor('tight','a',['Anyway', ',', 'my', 'pants', 'are', 'getting'],['every','day','.'])

In [160]:
[k for k,v in syn_freq.items() if v == max(syn_freq.values())]

AttributeError: 'str' object has no attribute 'items'

In [165]:
syn_freq

'taut'

In [ ]:
def predict_nearest(self,context : Context) -> str:
    lemma=context.lemma
    pos=context.pos
    synonyms=get_candidates(lemma, pos)
    result=dict()
    for w in synonyms:
        result[w]=self.model.similarity(lemma,w)
    result=[k for k,v in result.items() if v == max(result.values())][0]
    return result 

In [1]:
import transformers 
#import tensorflow as tf

ImportError: numpy>=1.17 is required for a normal functioning of this module, but found numpy==1.16.2.
Try: pip install transformers -U or pip install -e '.[dev]' if you're working with git main

In [170]:
!pip install transformers

In [14]:
import string
a=['Anyway', ',', 'my', 'pants', 'are', 'getting']
a.append('[MASK]')
a.extend(['every','day','.'])
s="".join([x if x in string.punctuation else " "+x for x in a])
s[1:]


>>> 
>>> 
>>> predictions = outputs[0]
>>>  # Sort in increasing order
>>> tokenizer.convert_ids_to_tokens(best_words[:10])
['stolen', 'wasted', 'worthless', 'dirty', 'yours', 'lost', 'money', 'worth', 'good']

'Anyway, my pants are getting [MASK] every day.'

In [ ]:
class BertPredictor(object):

    def __init__(self): 
        self.tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
        self.model = transformers.TFDistilBertForMaskedLM.from_pretrained('distilbert-base-uncased')

    def predict(self, context : Context) -> str:
        lemma=context.lemma
        pos=context.pos
        left_context=context.left_context
        right_context=context.right_context
        #obtain synonyms
        synonyms=get_candidates(lemma,pos)
        
        #step2
        left_context.append('[MASK]')
        left_context.extend(right_context)
        s="".join([x if x in string.punctuation else " "+x for x in left_context])
        s=s[1:]
        input_toks = self.tokenizer.encode(s)
        input_mat = np.array(input_toks).reshape((1,-1))
        mask_id=self.tokenizer.convert_ids_to_tokens(input_toks).index('[MASK]')
        
        
        #step3
        outputs = self.model.predict(input_mat)
        
        #step4
        predictions=outputs[0]
        best_words = np.argsort(predictions[0][mask_id])[::-1]
        words=self.tokenizer.convert_ids_to_tokens(best_words)
        
        result=dict()
        for w in synonyms:
            result[w]=words.index(w)
        
        best=[k for k,v in result.items() if v == min(result.values())][0]
        return best 

In [15]:
import transformers

ImportError: numpy>=1.17 is required for a normal functioning of this module, but found numpy==1.16.2.
Try: pip install transformers -U or pip install -e '.[dev]' if you're working with git main

In [16]:
import gensim

RuntimeError: Compiled extensions are unavailable. If you've installed from a package, ask the package maintainer to include compiled extensions. If you're building Gensim from source yourself, install Cython and a C compiler, and then run `python setup.py build_ext --inplace` to retry. 

In [20]:
result=dict()


0

In [27]:
lemma='slow'
pos='a'
synonyms=get_candidates(lemma, pos)
result=dict()
for w in synonyms:
    result[w]=1
result=[k for k,v in result.items() if v == max(result.values())][0]

In [28]:
result

'boring'